<a href="https://colab.research.google.com/github/guanyu1127/Programming-Language/blob/main/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

日常支出速算與分攤（作業一） 目標：從 Sheet 讀「消費紀錄」→ 計總額/分類小計/AA 分攤 → 寫回 Sheet Summary 分頁。 AI 點子（可選）：請模型總結本週花錢習慣與建議（例如「外食過多」）。 Sheet 欄位：date, category, item, amount, payer

In [20]:
# ============================================
# Colab 記帳自動化：讀「消費紀錄」→ 統計 → 寫回「Summary」
# 必要欄位：date, category, item, amount, payer
# ============================================

# ===== 0) 套件安裝 =====
!pip -q install gspread gspread-dataframe pandas google-auth google-auth-oauthlib pytz

# ===== 1) 基本設定 =====
SHEET_URL = "https://docs.google.com/spreadsheets/d/1zgd1GM9-Vi0JlwacPmbZF0Ci45QSX2t0EIZY1fmzfFE/edit?gid=0#gid=0"
DATA_SHEET_NAME = "消費紀錄"     # 來源分頁（請確保分頁名稱就是這個）
SUMMARY_SHEET_NAME = "Summary"   # 輸出分頁（不存在會自動建立）
TIMEZONE = "Asia/Taipei"

# ===== 2) Google 驗證與連線 =====
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

import pandas as pd
import numpy as np
import pytz
from datetime import datetime, timedelta, date

# ===== 3) 讀取資料分頁（穩健版）=====
ss = gc.open_by_url(SHEET_URL)

try:
    ws_data = ss.worksheet(DATA_SHEET_NAME)
except gspread.WorksheetNotFound:
    raise RuntimeError(f"找不到工作表『{DATA_SHEET_NAME}』。請先在試算表中新建該分頁，並在第一列輸入欄位：date, category, item, amount, payer")

values = ws_data.get_all_values()  # 取回含標題與資料的純字串陣列
if not values:
    raise RuntimeError(f"『{DATA_SHEET_NAME}』是空的。請在第 1 列建立標題：date, category, item, amount, payer，並從第 2 列開始輸入資料。")

raw_headers = values[0]
headers = [h.strip() for h in raw_headers]
df = pd.DataFrame(values[1:], columns=headers)  # 即便沒有資料列，columns 也會正確

# 容錯：把欄位名統一成小寫、去前後空白
df.columns = [c.strip().lower() for c in df.columns]

# ===== 4) 欄位與型態清理 =====
required_cols = ["date", "category", "item", "amount", "payer"]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise RuntimeError(f"缺少必要欄位：{missing}。請確認『{DATA_SHEET_NAME}』第 1 列為 {required_cols}")

# 若沒有任何資料列，直接提醒
if df.empty:
    raise RuntimeError("『消費紀錄』目前只有標題、沒有資料列。請先在第 2 列開始輸入至少一筆紀錄再執行。")

# 轉型（日期、金額）
def parse_date_safe(x):
    if pd.isna(x) or str(x).strip() == "":
        return pd.NaT
    try:
        return pd.to_datetime(x)
    except Exception:
        return pd.NaT

df["date"] = df["date"].apply(parse_date_safe)
df["amount"] = (
    df["amount"].astype(str).str.replace(",", "").str.strip()
).apply(lambda v: pd.to_numeric(v, errors="coerce"))

# 去除無效列
df = df.dropna(subset=["date", "category", "item", "amount", "payer"]).reset_index(drop=True)

if df.empty:
    raise RuntimeError("讀到的有效資料為空（可能有空白列或格式問題）。請檢查每欄都有值、金額為數字、日期可解析。")

# ===== 5) 總額與分類小計 =====
total_amount = float(df["amount"].sum())
by_category = (
    df.groupby("category", dropna=False)["amount"]
      .sum()
      .reset_index()
      .sort_values("amount", ascending=False)
)

# ===== 6) AA 分攤 =====
members = sorted(df["payer"].dropna().astype(str).unique().tolist())
n = len(members)
equal_share = total_amount / n if n > 0 else 0.0

paid_by = df.groupby("payer", dropna=False)["amount"].sum().reindex(members, fill_value=0).reset_index()
paid_by.columns = ["payer", "paid"]

aa_summary = paid_by.copy()
aa_summary["equal_share"] = round(equal_share, 2)
aa_summary["balance"] = (aa_summary["paid"] - equal_share).round(2)  # >0 應收、<0 應付

def settlement_plan(balances: pd.DataFrame):
    pos = balances[balances["balance"] > 0][["payer", "balance"]].copy()
    neg = balances[balances["balance"] < 0][["payer", "balance"]].copy()
    pos = pos.sort_values("balance", ascending=False).reset_index(drop=True)
    neg = neg.sort_values("balance").reset_index(drop=True)
    transfers = []
    i, j = 0, 0
    while i < len(neg) and j < len(pos):
        owe_name, owe_amt = neg.loc[i, "payer"], -float(neg.loc[i, "balance"])
        rec_name, rec_amt = pos.loc[j, "payer"], float(pos.loc[j, "balance"])
        pay = round(min(owe_amt, rec_amt), 2)
        if pay > 0:
            transfers.append({"from": owe_name, "to": rec_name, "amount": pay})
            neg.loc[i, "balance"] = -(owe_amt - pay)
            pos.loc[j, "balance"] = rec_amt - pay
        if neg.loc[i, "balance"] >= -1e-9:
            i += 1
        if pos.loc[j, "balance"] <= 1e-9:
            j += 1
    return pd.DataFrame(transfers)

transfer_df = settlement_plan(aa_summary)

# ===== 7) 本週消費摘要（規則版）=====
tz = pytz.timezone(TIMEZONE)
today_local = datetime.now(tz).date()
weekday = today_local.weekday()               # Mon=0
week_start = today_local - timedelta(days=weekday)
week_end = week_start + timedelta(days=6)

df_week = df[(df["date"].dt.date >= week_start) & (df["date"].dt.date <= week_end)]
week_total = float(df_week["amount"].sum()) if not df_week.empty else 0.0

def rule_based_summary(df_all: pd.DataFrame, df_w: pd.DataFrame) -> str:
    if df_w.empty:
        return f"本週（{week_start} ~ {week_end}）尚無消費紀錄。"
    top_cat = df_w.groupby("category")["amount"].sum().sort_values(ascending=False)
    tips = []
    if not top_cat.empty:
        first = str(top_cat.index[0])
        share = float(top_cat.iloc[0] / top_cat.sum() * 100)
        tips.append(f"最高支出類別「{first}」占比 {share:.1f}%。")
    # 簡單提醒：高單筆
    high = df_w.sort_values("amount", ascending=False).head(1)
    if not high.empty and float(high.iloc[0]["amount"]) >= 1000:
        tips.append(f"最大單筆「{high.iloc[0]['item']}」{int(high.iloc[0]['amount'])}。")
    return f"本週（{week_start} ~ {week_end}）總支出 {week_total:.0f}。 " + " ".join(tips)

weekly_summary_text = rule_based_summary(df, df_week)

# ===== 8) 要寫回的表格 =====
overview_rows = [
    ["指標", "數值"],
    ["總支出", round(total_amount, 2)],
    ["成員（AA）", ", ".join(members)],
    ["平均分攤(每人)", round(equal_share, 2)],
    ["週期(本週)", f"{week_start} ~ {week_end}"],
]

cat_table = [["category", "subtotal"]] + [[str(r["category"]), round(float(r["amount"]), 2)] for _, r in by_category.iterrows()]
aa_table = [["payer", "paid", "equal_share", "balance"]] + [
    [str(r["payer"]), round(float(r["paid"]), 2), round(float(r["equal_share"]), 2), round(float(r["balance"]), 2)]
    for _, r in aa_summary.iterrows()
]
transfer_table = [["from", "to", "amount"]] + (
    [[str(r["from"]), str(r["to"]), round(float(r["amount"]), 2)] for _, r in transfer_df.iterrows()]
    if not transfer_df.empty else [["—", "—", 0]]
)
summary_paragraph = [["Weekly Summary"], [str(weekly_summary_text)]]

# ===== 9) 寫回 Summary（修正版 write_block：型別轉原生 + 新式參數）=====
try:
    ws_summary = ss.worksheet(SUMMARY_SHEET_NAME)
except gspread.WorksheetNotFound:
    ws_summary = ss.add_worksheet(title=SUMMARY_SHEET_NAME, rows=200, cols=20)

ws_summary.clear()

def _to_native(v):
    """把 pandas/NumPy/Timestamp 轉成 gspread 可接受的原生型別或字串。"""
    import json
    # None/NaN -> 空字串
    try:
        if pd.isna(v):
            return ""
    except Exception:
        pass
    # numpy/pandas 數值 -> 原生
    if isinstance(v, (np.integer,)):
        return int(v)
    if isinstance(v, (np.floating,)):
        return float(v)
    # 日期/時間 -> YYYY-MM-DD
    if isinstance(v, (pd.Timestamp, datetime, date)):
        return pd.to_datetime(v).strftime("%Y-%m-%d")
    # 可序列化就原樣，不行就轉字串
    try:
        json.dumps(v)
        return v
    except Exception:
        return str(v)

def write_block(start_row, start_col, values):
    conv = [[_to_native(x) for x in row] for row in values]
    end_row = start_row + len(conv) - 1
    end_col = start_col + len(conv[0]) - 1
    rng = gspread.utils.rowcol_to_a1(start_row, start_col) + ":" + gspread.utils.rowcol_to_a1(end_row, end_col)
    # 新版建議的命名參數順序
    ws_summary.update(values=conv, range_name=rng)
    return end_row + 2

row = 1
row = write_block(row, 1, [["=== Overview ==="]]);            row = write_block(row, 1, overview_rows)
row = write_block(row, 1, [["=== Category Subtotals ==="]]);  row = write_block(row, 1, cat_table)
row = write_block(row, 1, [["=== AA Split ==="]]);            row = write_block(row, 1, aa_table)
row = write_block(row, 1, [["=== Suggested Transfers ==="]]); row = write_block(row, 1, transfer_table)
row = write_block(row, 1, [["=== Weekly Summary (Auto) ==="]]); row = write_block(row, 1, summary_paragraph)

print("✅ 已完成：讀取消費紀錄 → 統計 → 寫回 Summary。請到試算表查看結果！")


✅ 已完成：讀取消費紀錄 → 統計 → 寫回 Summary。請到試算表查看結果！
